#Web scraping del sito della collana di libri Adelphi 
Una delle task più comuni per cui viene impiegato Python è il così detto **web scraping**. Il web scraping consiste nell'automatizzare l'ottenimento di alcune informazioni dal web tramite un codice e di utilizzarle di conseguenza. La qualità e la semplicità nell'ottenimento delle informazioni è dato solitamente dalla struttura della fonte: alcuni siti sono molto più semplici da navigare mentre altri possono essere più complessi. Per questo motivo il web scraping è spesso molto complesso. 
In questo esercizio vi verrà fornito uno scraper già fatto in grado di ottenere informazioni dal catalogo online della nota collana di **libri Adelphi** (https://www.adelphi.it/): calandovi nei panni di un avido lettore e collezionista di libri, il vostro obiettivo sarà di utilizzarlo per digitalizzare interamente la vostra collezione di libri, in modo da poter velocemente controllare l'inventario dei vostri libri ed alcune informazioni a riguardo con un semplice comando ogni volta che lo vorrete.

In [51]:
# IMPORTANTE
# questa cella di codice serve ad installare nel vostro Python due tra i principali strumenti di web scraping: beautiful soup e requests.
# qualora di fossero problemi con la loro installazione o import non esitate a contattarci
!pip install requests
!pip install beautifulsoup4

# Implementazione dello scraper
Come anticipato, non vi sarà richiesto di implementare lo scraper. Tuttavia gli interessati possono trovare informazioni su come utilizzare al meglio le due librerie per lo scraping *requests* e *beautifulsoup* e cercare di comprendere il codice della classe AdelphiScraper. Se non siete tra questi potete saltare alla prossima sezione.
Qui qualche risorsa:

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

https://requests.readthedocs.io/en/master/user/quickstart/ 

In [52]:
import requests as req
import bs4 as bs



In [53]:
class AdelphiScraper():
  def __init__(self):
    self.catalog      = 'https://www.adelphi.it/catalogo/'
    self.subject      = 'https://www.adelphi.it/catalogo/materia/'
    self.subjects_map = self.init_subjects()

  def init_subjects(self):
    resp  = req.get(self.catalog)
    soup  = bs.BeautifulSoup(resp.text, "html.parser")
    items = soup.find_all('ul', class_='subject')
    ret = dict()
    for i in items:
      for j in i:
        ret.update({int(j.a['href'].split('/')[-1]) : j.text})
    return ret

  def get_subject_list(self):
    return self.subjects_map
  
  def get_subject_books(self, subject_code):
    page_found = True
    suffix = ''
    page_counter = 1
    books = dict()
    isbns = dict()
    while page_found:
      try:
        resp = req.get(self.subject + str(subject_code) + suffix, allow_redirects = False)
        soup = bs.BeautifulSoup(resp.text, "html.parser")
        if not len(soup):
          raise ValueError()
        book_infos, isbn_maps = self.extract_books(soup)
        books.update(book_infos)
        isbns.update(isbn_maps)
        page_counter += 1
        suffix = '/p' + str(page_counter)
      except:
        page_found = False
    return books, isbns

  def extract_books(self, soup):
    items = soup.find_all('div', class_='list_impressum')
    items2 = soup.find_all('div', class_='data' )
    items3 = soup.find_all('div', class_ = "abstract hidden-xs")
    isbn_map  = dict()
    book_info = dict()
    counter = 0
    for i in items:
      isbn_map.update({i.a['href'].split('/')[-1] : i.a.text}) #isbn : titolo
      book_info.update({i.a.text : {'Autore': i.div.text, \
                                    'Anno': items2[counter].text.split(' ')[0].split(',')[0], \
                                    'Prezzo Originale' : i.span.text.split(' ')[1].replace(',','.'), \
                                    'Sconto': i.span.text.split(' ')[2], \
                                    'Pagine' :items2[counter].text.split(' ')[2].split(',')[0].split('-')[-1], \
                                    'Abstract': items3[counter].text}})
      counter += 1
    return book_info, isbn_map

#Utilizzo della classe AdelphiScraper

*   La classe AdelphiScraper non richiede alcun argomento di inizializzazione.
*   Essendo un WebScraper richiede ovviamente la connessione ad internet.
*   Essa contiene al suo interno diversi attributi e metodi, i metodi che vi serviranno sono:
    * `get_subject_list()` : questo metodo restituisce un dizionario contenente tutte le categorie della collana di libri Adelphi e il codice numerico a cui sono associate sul sito.
    * `get_subject_books(category_code)` : questo metodo riceve il codice numerico (**come intero**) di una categoria e restituisce due dizionari: 
        * Il primo ha come chiave i titoli dei libri appartenenti alla categoria e come valori svariati dati ad essi inerenti (Autore, prezzo, anno di stampa dell'ultima edizione, etc.)
        * Il secondo ha come chiavi gli ISBN (codice univoco diverso da libro a libro) e come valori i titoli dei libri associati.




In [54]:
## Come prima cosa istanziate un oggetto di classe AdelphiScraper e provate ad utilizzare i due metodi sopra descritti per osservarne l'output,
## questa operazione è molto importante al fine di capire bene quali sono le strutture dati a vostra dispozione.
## ATTENZIONE : osservate bene quali sono i tipi dei dati contenuti nelle strutture che riceverete in output, più avanti avrete bisogno di farvi operazioni o conversioni.

Ad_Scraper = AdelphiScraper()

diz_subject = Ad_Scraper.get_subject_list()

diz_books = Ad_Scraper.get_subject_books(2)



In [55]:
def get_title_from_ISBN(ISBN): #LENTISSIMO, non saprei come velocizzarlo
    for i in range(1,max(Ad_Scraper.get_subject_list())):
        diz_temp = Ad_Scraper.get_subject_books(i)
        if  ISBN in diz_temp[1].keys():
            return diz_temp[1][ISBN], i
            
def get_infos_from_title(title, index):
    diz_temp = Ad_Scraper.get_subject_books(index)
    author = diz_temp[0][title]['Autore']
    year = diz_temp[0][title]['Anno']
    abstract = diz_temp[0][title]['Abstract']
    pages = diz_temp[0][title]['Pagine']
    
    return year, author, abstract, pages
    
    
    

class Book():
    def __init__(self,ISBN):
        self.isbn = ISBN
        [self.title, index] = get_title_from_ISBN(ISBN)
        [year, author, abstract, pages] = get_infos_from_title(self.title, index)
        self.print_year = year
        self.author = author
        self.abstract = abstract
        self.pages = pages
        self.category = Ad_Scraper.get_subject_list()[index]
        self.index = index  #mi serve per risalire più velocemente alla categoria
        
    def abstract_len(self):
        return len(self.abstract)
        
    def get_whole_price(self):
        temp = Ad_Scraper.get_subject_books(self.index)[0] #accedo alle info relative (contenute nello scraper)
        return temp[self.title]['Prezzo Originale']
    
    def get_net_price(self):
        temp = Ad_Scraper.get_subject_books(self.index)[0] #accedo alle info relative (contenute nello scraper)
        num = temp[self.title]['Sconto'].replace('%', '')
        num = 1+float(num)/100  #percentuale di prezzo che dovrò pagare
        return float(temp[self.title]['Prezzo Originale'])*num
    
    def print_book_infos(self):
        print("Il libro intitolato",self.title,", lungo ",self.pages," pagine, dell'anno ",self.print_year,", dell'autore ",self.author,"può essere descritto dal seguente abstract: \n",self.abstract)
        #non ho messo tutto ma l'idea è quella
        
tries = Book('9788845931536')
        




In [57]:
tries.get_net_price()

26.599999999999998

# Creazione di una libreria digitale
Richieste dell'esercizio:

*   Implementare la classe `Book(ISBN)`: questa classe dovrà essere inizializzata soltanto dall'ISBN di un libro, utilizzando AdelphiScraper dovrà essere in grado di inizializzarsi con i seguenti attributi:
    *   `self.ISBN` : ISBN del libro.
    *   `self.title` : il titolo del libro.
    *   `self.print_year` : l'anno dell'ultima ristampa.
    *   `self.author` : l'autore del libro.
    *   `self.abstract` : abstract del libro.
    *   `self.pages` : numero di pagine del libro.
    *   `self.category` : categoria a cui appartiene il libro.

  **Suggerimento**: tutte queste informazioni devono essere reperite con varie chiamate ai metodi di AdelphiScraper e tramite trasformazioni alle strutture dati ricevute, si consiglia caldamente di chiamare AdelphiScraper esternamente e creare una struttura dati adeguata a cercare tutte le informazioni velocemente dato **soltanto** l'ISBN di un libro, a quel punto la classe Book potrà attingere da lì senza connettersi allo scraper in ogni sua istanza.

  Dovrà inoltre possedere i seguenti metodi:
    *   `get_whole_price(self)` : restituirà il prezzo lordo del libro usando AdelphiScraper.
    *   `get_net_price(self)` : restituirà il prezzo netto del libro usando AdelphiScraper.
    *   `print_book_infos(self)` : stamperà in output le varie informazioni sul libro.
    *   `abstract_len` : restituisce lunghezza dell'abstract del libro

* Vi è richiesto di digitalizzare le informazioni di alcuni libri, creare una classe chiamata `MyLibrary(ISBN_list)` che possa essere inizializzata soltanto con una lista di ISBN. In modo simile alla classe `Book(ISBN)` questa dovrà far uso di AdelphiScraper per collezionare le informazioni di tutti i libri in essa contenuti, si suggerisce l'utilizzo di dizionari e oggetti di tipo Book come strutture dati replicando gli stessi attributi e metodi (in questi ultimi si possa specificare il libro del quale siamo interessati al prezzo, all'abstract etc.).
La classe dovrà inoltre avere i seguenti metodi:
    * `insert_book(self, ISBN) `: inserisca nella libreria l'oggetto di tipo Book corrispondente all'ISBN passato.
    * `remove_book(self, ISBN) `: rimuova dall libreria l'oggetto di tipo Book corrispondente all'ISBN passato.
    * `get_library_value(self) `: restituirà il valore totale (dato dai prezzi) dei libri contenuti nella libreria, sia lordo che al netto degli sconti.
    * `get_total_pages(self) ` : restituirà il numero totale di pagine dei libri nella nostra libreria
    * `get_categories_number(self) ` : restituirà il numero di categorie differenti che sono presenti nella nostra libreria.

* Introduciamo il concetto di libri "vicini": diciamo che due libri sono vicini se appartengono alla stessa categoria **oppure** il loro ISBN modulo 3 è lo stesso: se ad esempio due libri hanno come ISBN 134434**6** e ISBN 294823**9** questi avranno entrambi ISBN modulo 3 uguale a 0 e saranno vicini.
    * Si inserisca nella classe `Book(ISBN)` il metodo `is_near(self, book_2)` che restituisca `True` se l'oggetto di tipo Book passato in input è vicino al libro rappresentato dall'oggetto Book che chiama il metodo, `False` altrimenti.
```
b1 = Book(123456)
b2 = Book(123456789)
b1.is_near(b2)
>> True
```
    * Si inserisca nella classe `MyLibrary(ISBN_list)` il metodo `minimum_path(self, start_ISBN, end_ISBN) ` che riceve in input due ISBN appartenenti alla libreria, uno iniziale ed uno finale, e restituisca la lista ordinata degli ISBN dei libri facenti parte della libreria da cui bisogna passare per arrivare da quello iniziale a quello finale. Tuttavia, è possibile passare da un libro ad un altro solo se sono vicini, questa funzione deve restituire la sequenza di passaggi di lunghezza **più breve** per arrivare a destinazione.
    * Si inserisca nella classe `MyLibrary(ISBN_list)` il metodo `minimum_cost_path(self, start_ISBN, end_ISBN) `, una versione alternativa del metodo precedente che invece che il cammino di lunghezza minore restituisce quello con il costo **netto** totale dei libri facenti parte del cammino minore possibile.

Si usino i seguenti ISBN come caso prova delle varie implementazioni:

Cose che potrebbero esservi utili: 
  * Attributi dei dizionari: .keys(), .values(), ...
  * Teoria dei grafi
  